In [3]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
base_path = '/content/gdrive/My Drive/Colab Notebooks'

In [5]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
train_input_data = 'nsmc_train_input.npy'
train_label_data = 'nsmc_train_label.npy'
data_configs = 'data_configs.json'

input_data = np.load(open(data_in_path + train_input_data, 'rb'))
label_data = np.load(open(data_in_path + train_label_data, 'rb'))
prepro_configs = json.load(open(data_in_path + data_configs, 'r'))

In [8]:
test_split = 0.1
seed = 100
batch_size = 50
num_epochs = 16

input_train, input_eval, label_train, label_eval = train_test_split(input_data,
                                                                    label_data,
                                                                    test_size=test_split,
                                                                    random_state=seed)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
dataset = dataset.shuffle(buffer_size=len(input_train))
dataset = dataset.batch(batch_size)

validation_dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
validation_dataset = validation_dataset.shuffle(buffer_size=len(input_eval))
validation_dataset = validation_dataset.batch(batch_size)

In [12]:
from tensorflow.keras import layers

In [43]:
class Model(tf.keras.Model):
    
    def __init__(self, **kargs):
        super(Model, self).__init__(name='conv')
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                     output_dim=kargs['embedding_size'])
        self.conv_list = [layers.Conv1D(filters=kargs['num_filters'],
                                   kernel_size=kernel_size,
                                   padding='valid',
                                   activation=tf.keras.activations.relu,
                                   kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
                     for kernel_size in [3,4,5]]
        self.pooling = layers.GlobalMaxPooling1D()
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                           activation=tf.keras.activations.relu,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
        self.fc2 = layers.Dense(units=kargs['output_dimension'],
                           activation=tf.keras.activations.sigmoid,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [44]:
kargs = {'vocab_size': prepro_configs['vocab_size'],
        'embedding_size': 128,
        'num_filters': 100,
        'dropout_rate': 0.5,
        'hidden_dimension': 250,
        'output_dimension':1}

In [48]:
model = Model(**kargs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(dataset, epochs=num_epochs,
         validation_data=validation_dataset)

Epoch 1/5
2700/2700 [==============================] - 133s 49ms/step - loss: 0.3955 - accuracy: 0.8170 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
2700/2700 [==============================] - 129s 48ms/step - loss: 0.3050 - accuracy: 0.8696 - val_loss: 0.3331 - val_accuracy: 0.8547
Epoch 3/5
2700/2700 [==============================] - 130s 48ms/step - loss: 0.2494 - accuracy: 0.8976 - val_loss: 0.3345 - val_accuracy: 0.8568
Epoch 4/5
2700/2700 [==============================] - 130s 48ms/step - loss: 0.2098 - accuracy: 0.9144 - val_loss: 0.3550 - val_accuracy: 0.8538
Epoch 5/5
2700/2700 [==============================] - 131s 49ms/step - loss: 0.1781 - accuracy: 0.9282 - val_loss: 0.3834 - val_accuracy: 0.8517


In [49]:
test_input_data = 'nsmc_test_input.npy'
test_label_data = 'nsmc_test_label.npy'

test_input_data = np.load(open(data_in_path + test_input_data, 'rb'))
test_label_data = np.load(open(data_in_path + test_label_data, 'rb'))

In [51]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
test_dataset = test_dataset.batch(BATCH_SIZE)

In [53]:
test_output = model.predict(test_dataset)

In [54]:
test_output = np.array(test_output)

In [55]:
test_output

array([[0.9836465 ],
       [0.54142475],
       [0.26841763],
       ...,
       [0.7737855 ],
       [0.00120722],
       [0.0788133 ]], dtype=float32)